In [4]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# for the model
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, GRU, RepeatVector, TimeDistributed, Reshape
from keras import optimizers
from keras.callbacks import ReduceLROnPlateau, EarlyStopping

Using TensorFlow backend.


In [114]:
# load files from pre-processing

#X = np.load('X.npy') #rainpredictions, hour of day, month of year, holiday, ones
#y = np.load('y.npy')

X_dry = np.load('X_dry.npy',allow_pickle=True)
y_dry = np.load('Y_dry.npy',allow_pickle=True)
X_rain = np.load('X_rain.npy',allow_pickle=True)
y_rain = np.load('Y_rain.npy',allow_pickle=True)
TimeHour = pd.read_csv('Timehour_wet.csv')

array([[554.1805633333325, 0.0, 0.0, ..., 0.0, 1.0, 1.0],
       [512.610779583333, 1.0, 0.0, ..., 0.0, 0.0, 1.0],
       [495.14290208333324, 0.0, 1.0, ..., 0.0, 0.0, 1.0],
       ...,
       [113.38789088512496, 0.0, 0.0, ..., 0.0, 0.0, 1.0],
       [77.43711137416668, 0.0, 0.0, ..., 0.0, 0.0, 1.0],
       [163.3501073891665, 0.0, 0.0, ..., 0.0, 0.0, 1.0]], dtype=object)

In [108]:
# split dataset in test and train

def ReshapeXData(dataframe):
    n_hours = 24
    n_features = dataframe.shape[1]
    dataframe_shaped = np.reshape(dataframe,(-1, n_hours, n_features))
    return dataframe_shaped

def ReshapeYData(dataframe):
    n_hours = 24
    n_features = 1
    dataframe_shaped = np.reshape(dataframe,(-1, n_hours, n_features))
    return dataframe_shaped


X_train, X_test, y_train, y_test = train_test_split(X_rain, y_rain, test_size=0.19847328244, random_state=42)

X_shaped = ReshapeXData(X_rain)
X_train = ReshapeXData(X_train)
X_test = ReshapeXData(X_test)
y_train = ReshapeYData(y_train)
y_test = ReshapeYData(y_test)
#X_train_gru = X[0:10800,:]
#X_test_gru = X[10801:13609,:]
#Y_train_gru = y[0:10800]
#Y_test_gru = y[10801:13609]


#X_train_gru_shaped=ReshapeXData(X_train_gru)
#X_test_gru_shaped = ReshapeXData(X_test_gru)
#Y_train_gru_shaped = ReshapeYData(Y_train_gru)
#Y_test_gru_shaped = ReshapeYData(Y_test_gru)
#X_shaped = ReshapeXData(X[0:13608,:])
#Y_shaped = ReshapeYData(y[0:13608])

In [110]:
# basic model

from keras.layers import Dense

lr=0.0002
epochs=100
batch_size=32
validation_split=0.1

print('Build model...')

n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train.shape[1]
#n_timesteps, n_features, n_outputs = X_train_gru.shape[0], X_train_gru.shape[1], Y_train_gru.shape[0]
# reshape output into [samples, timesteps, features]

model = Sequential()
#model.add(Reshape((1, X_train_gru.shape[1])))
model.add(GRU(200, activation='relu', input_shape=(n_timesteps, n_features)))
#model.add(GRU(200, activation='tanh', input_shape=(1,X_train_gru.shape[1])))
model.add(RepeatVector(n_outputs))
model.add(GRU(200, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(100, activation='relu')))
model.add(TimeDistributed(Dense(1)))
opt=optimizers.Adam(learning_rate=lr)
model.compile(loss='mae', optimizer=opt)

model.fit(
        X_train,
        y_train,
        epochs=epochs,
        batch_size=batch_size,
        verbose=1,
        validation_split=0.2,
        callbacks=[
            ReduceLROnPlateau(
                verbose=1)])
    
    
#model = Sequential()
#model.add(Dense(1, input_dim=X.shape[1]))
#model.compile(optimizer = optimizers.Adam(lr=lr), loss='mean_squared_error')
#model.fit(X_train, y_train,
#          validation_split=validation_split,
#          epochs=epochs,
#          batch_size=batch_size,
#          shuffle=False)

Build model...
Train on 84 samples, validate on 21 samples
Epoch 1/100
84/84 [==============================] - 1s 7ms/step - loss: 223.3351 - val_loss: 221.5797
Epoch 2/100
84/84 [==============================] - 0s 2ms/step - loss: 217.0239 - val_loss: 214.7480
Epoch 3/100
84/84 [==============================] - 0s 2ms/step - loss: 210.3318 - val_loss: 207.3567
Epoch 4/100
84/84 [==============================] - 0s 2ms/step - loss: 202.9140 - val_loss: 199.8026
Epoch 5/100
84/84 [==============================] - 0s 2ms/step - loss: 195.2464 - val_loss: 191.9962
Epoch 6/100
84/84 [==============================] - 0s 2ms/step - loss: 187.2317 - val_loss: 184.3537
Epoch 7/100
84/84 [==============================] - 0s 2ms/step - loss: 179.2071 -

In [111]:
# make a prediction to test how well the model preforms
y_predicted = model.predict(X_test)

In [ ]:
### LZS: WE HAVE TO IMPLEMENT HERE SOME METRICS AND A LOT OF PLOTS SO THAT WE CAN SEE HOW WELL THE MODEL PERFORMS
### LZS: AND ALSO TO BE ABLE TO FIND NEW FEATURES TO IMPROVE THE MODEL
### LZS: AND TO EVALUATE IF THE MODEL INDEED IMPROVES WHEN ADDING MORE FEATURES/ OTHER LAYERS

In [112]:
# show the inputs, predicted outputs and real outputs
for i in range(len(X_test[0])):
        for j in range(len(X_test[1])):
            print("Predicted=%s, Original=%s" % (y_predicted[i][j], y_test[i][j]))

Predicted=[72.248436], Original=[84.4234122904167]
Predicted=[86.923], Original=[862.5090558333333]
Predicted=[88.00898], Original=[365.5664364264861]
Predicted=[88.88442], Original=[336.159522455139]
Predicted=[90.23452], Original=[146.23547872013904]
Predicted=[91.5743], Original=[63.33341904555554]
Predicted=[92.92094], Original=[498.6270929166664]
Predicted=[94.30749], Original=[514.8907244444447]
Predicted=[95.76746], Original=[513.2867190277781]
Predicted=[97.24717], Original=[43.21729329622221]
Predicted=[98.77364], Original=[35.78967036840332]
Predicted=[100.38198], Original=[120.85732251188983]
Predicted=[102.02639], Original=[35.80649369249831]
Predicted=[103.73035], Original=[10.226580554208333]
Predicted=[105.49932], Original=[91.94390274458337]
Predicted=[107.35139], Original=[83.82949924069443]
Predicted=[109.34038], Original=[104.96816865428814]
Predicted=[111.46469], Original=[496.9309556388885]
Predicted=[113.796715], Original=[151.64836002888876]
Predicted=[116.36996]

In [115]:
# Create a dataframe with a prediction for all datapoints for the csv
y_predicted_all = model.predict(X_shaped)
y_pred_all = np.reshape(y_predicted_all, (-1, 1))
print(y_pred_all.shape)
y_pred_all = pd.DataFrame(data=y_pred_all)
print(TimeHour.shape)
my_df = pd.DataFrame({'Time': TimeHour.loc[:,'TimeHour'].tolist(), 'pred_y': y_pred_all.loc[:,0].tolist()})
my_df.to_csv('WetPredictions.csv')


(3144, 1)
(3144, 2)


In [58]:
def fill_with_average(df, previous, following):
    value_new = list(df.loc[previous])
    value_previous = list(df.loc[previous])
    value_following = list(df.loc[following])
    value_new[0]= (value_previous[0]+value_following[0])/2
    date = df.loc[previous, 'TimeHour2']
    new_date = date + timedelta(hours=1)
    value_new[40]=new_date
    value_new[date.hour]=0.0
    value_new[new_date.hour]=1.0
    #value_new[43]=1.0
    df_length = len(df)
    df.loc[df_length] = value_new
    df = df.sort_values(by= "TimeHour2")
    df = df.reset_index()
    df = df.drop(columns=["index"])
    return df

def Y_fill_with_average(df, previous, following):
    value_new = list(df.loc[previous])
    value_previous = list(df.loc[previous])
    value_following = list(df.loc[following])
    value_new[0]= (value_previous[0]+value_following[0])/2
    date = df.loc[previous, 'TimeHour2']
    new_date = date + timedelta(hours=1)
    value_new[2]=new_date
    #value_new[date.hour]=0.0
    #value_new[new_date.hour]=1.0
    #value_new[43]=1.0
    df_length = len(df)
    df.loc[df_length] = value_new
    df = df.sort_values(by= "TimeHour2")
    df = df.reset_index()
    df = df.drop(columns=["index"])
    return df



In [65]:
df_test = pd.DataFrame(data=X)
df_test['TimeHour'] = TimeHour.loc[:, 'TimeHour']
df_test['TimeHour2'] = 'NA'
df_test['TimeHour2'] = pd.to_datetime(df_test['TimeHour'], format='%Y-%m-%d %H:%M:%S')
df_test['Diff'] = 'NA'
df_test['Before']="NA"
for x in range(1, df_test.shape[0]):
    before = df_test.loc[x - 1, 'TimeHour2']
    df_test.loc[x, 'Before'] = before
    now = df_test.loc[x, 'TimeHour2']
    diff = now - before
    df_test.loc[x, 'Diff'] = diff
df_test
df_test = df_test.drop(0)
df_test['diff_hours'] = [x.total_seconds() / 3600 for x in df_test['Diff']]



df_test = df_test.drop([1141,1142])
df_test = df_test.reset_index()
df_test = df_test.drop(columns=["index"])
df_test = df_test.drop([655,656,657,658,659,660,661,662,663,664,665,666,667,668,669])
df_test = df_test.reset_index()
df_test = df_test.drop(columns=["index"])
df_test = df_test.drop([641,642,643,644,645,646,647,648,649,650,651,652,653,654,655])
df_test = df_test.reset_index()
df_test = df_test.drop(columns=["index"])
df_test = df_test.drop([1480])
df_test = df_test.drop([7900,7901,7902,7903,7904,7905])
df_test = df_test.reset_index()
df_test = df_test.drop(columns=["index"])
df_test[df_test['diff_hours']>1]


,0,TimeHour,TimeHour2,Diff,Before,diff_hours
550,47.628271,2018-01-25 14:00:00,2018-01-25 14:00:00,1 days 04:00:00,2018-01-24 10:00:00,28.0
662,0.000000,2018-02-05 16:00:00,2018-02-05 16:00:00,4 days 05:00:00,2018-02-01 11:00:00,101.0
1037,76.974656,2018-02-23 09:00:00,2018-02-23 09:00:00,2 days 03:00:00,2018-02-21 06:00:00,51.0
1110,91.311921,2018-02-28 13:00:00,2018-02-28 13:00:00,0 days 04:00:00,2018-02-28 09:00:00,4.0
1479,35.789604,2018-04-05 20:00:00,2018-04-05 20:00:00,20 days 23:00:00,2018-03-15 21:00:00,503.0
1661,0.000000,2018-04-13 16:00:00,2018-04-13 16:00:00,0 days 06:00:00,2018-04-13 10:00:00,6.0
1774,45.289669,2018-04-18 17:00:00,2018-04-18 17:00:00,0 days 09:00:00,2018-04-18 08:00:00,9.0
1793,45.334073,2018-04-20 12:00:00,2018-04-20 12:00:00,1 days 01:00:00,2018-04-19 11:00:00,25.0
2133,46.871539,2018-05-04 18:00:00,2018-05-04 18:00:00,0 days 03:00:00,2018-05-04 15:00:00,3.0
3747,43.357102,2018-07-11 01:00:00,2018-07-11 01:00:00,0 days 02:00:00,2018-07-10 23:00:00,2.0


In [66]:
x = 0
while x < 13000:
    #if df_test.loc[x, 'diff_hours'] != 1.0 and df_test.loc[x, "diff_hours"] != 24.0 and df_test.loc[x, 'diff_hours'] != 96.0 and df_test.loc[x, 'diff_hours'] != 48.0 and df_test.loc[x, 'diff_hours'] != 480.0:
    if df_test.loc[x,'diff_hours'] %24!=0.0 and df_test.loc[x,'diff_hours'] != 1.0:
        print(x)
        df_test.loc[x, 'diff_hours'] = df_test.loc[x, 'diff_hours'] - 1
        df_test = Y_fill_with_average(df_test, x-1, x)
        x = x - 1
    else:
        x = x + 1

print ('Missing values approximated')
df_test[df_test['diff_hours']>1]

550
551
552
553
666
667
668
669
670
1046
1047
1048
1122
1123
1124
1494
1495
1496
1497
1498
1499
1500
1501
1502
1503
1504
1505
1506
1507
1508
1509
1510
1511
1512
1513
1514
1515
1516
1699
1700
1701
1702
1703
1817
1818
1819
1820
1821
1822
1823
1824
1844
2185
2186
3801
4861
4862
5325
5326
5327
6682
6683
6684
6685
6686
6687
6688
10091
12717
Missing values approximated


,0,TimeHour,TimeHour2,Diff,Before,diff_hours
554,47.628271,2018-01-25 14:00:00,2018-01-25 14:00:00,1 days 04:00:00,2018-01-24 10:00:00,24.0
671,0.000000,2018-02-05 16:00:00,2018-02-05 16:00:00,4 days 05:00:00,2018-02-01 11:00:00,96.0
1049,76.974656,2018-02-23 09:00:00,2018-02-23 09:00:00,2 days 03:00:00,2018-02-21 06:00:00,48.0
1517,35.789604,2018-04-05 20:00:00,2018-04-05 20:00:00,20 days 23:00:00,2018-03-15 21:00:00,480.0
1845,45.334073,2018-04-20 12:00:00,2018-04-20 12:00:00,1 days 01:00:00,2018-04-19 11:00:00,24.0


In [68]:

df_test = df_test.drop(['Diff', 'TimeHour2', 'diff_hours', 'Before'],axis=1)
df_test = df_test.drop([13632,13633,13634,13635,13636,13637,13638,13639,13640])

print(df_test.shape)
df_test

(13632, 2)


,0,TimeHour
0,502.995390,2018-01-01 13:00:00
1,501.561763,2018-01-01 14:00:00
2,267.722037,2018-01-01 15:00:00
3,164.684623,2018-01-01 16:00:00
4,415.606068,2018-01-01 17:00:00
...,...,...
13627,95.319013,2019-08-25 10:00:00
13628,158.127773,2019-08-25 11:00:00
13629,107.449515,2019-08-25 12:00:00
13630,184.320927,2019-08-25 13:00:00


In [69]:
np.save('Y_cleaned.npy', df_test)